# <center>Organizaci&oacute;n de Datos</center>
#### <center>C&aacute;tedra Ing. Rodriguez, Juan Manuel </center>

## <center>Trabajo Práctico 2:</center>
### <center> Propiedades en Venta </center>
### <center>  Grupo 18</center>


#### **Integrantes:**

* Analía Acevedo
* Santiago Daniel Marczewski Neudeck
* Lucia Belén Napoli
* Brayan Ricaldi

In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sklearn.preprocessing as skp

from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import (
    KBinsDiscretizer,
    LabelEncoder,
    MinMaxScaler,
    Normalizer,
    OneHotEncoder,
    OrdinalEncoder,
    PowerTransformer,
    RobustScaler,
    StandardScaler,
)

import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)
from preprocessing import preprocesamiento

from sklearn.model_selection import RandomizedSearchCV
from joblib import dump, load

pd.options.display.max_columns = None

from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.datasets import make_classification
from collections import defaultdict
from sklearn.model_selection import GridSearchCV

import plotly.graph_objects as go
%matplotlib inline


from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold

from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt
import visualkeras
from keras.wrappers.scikit_learn import KerasClassifier

np.random.seed(1)
tf.random.set_seed(1)

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

Descargo los datos para el modelo 1

In [2]:
_x_train_model1 = pd.read_csv('./DataSet/x_train')
_x_train_model1.drop(columns=['Unnamed: 0'],inplace=True)
_y_train_model1 = pd.read_csv('./DataSet/y_train')
_y_train_model1.drop(columns=['Unnamed: 0'],inplace=True)
_x_test_model1 = pd.read_csv('./DataSet/x_test')
_x_test_model1.drop(columns=['Unnamed: 0'],inplace=True)
_y_test_model1 = pd.read_csv('./DataSet/y_test')
_y_test_model1.drop(columns=['Unnamed: 0'],inplace=True)
_y_train_model1.rename(columns={'0':'pxm2'}, inplace=True)
_y_test_model1.rename(columns={'0':'pxm2'}, inplace=True)

# Redes neuronales

Construir dos modelos de redes neuronales, uno para regresión y otro para clasificación,
considerando los datasets creados en el TP1. Mejorar estos modelos de redes
neuronales a través de la búsqueda de arquitectura e hiperparámetros adecuados. Se
pide:

● Regresión: predecir el precio de la propiedad y utilizar como métrica de
evaluación el error cuadrático medio.

● Clasificación: predecir el atributo tipo_precio creado en el TP 1 y utilizar como
métricas precisión, recall y F1-Score.


# Modelo de Redes Neuronales para regresión

Los hiperparametros a buscar son los siguientes:

- **Función de activación:** relu, tanh
- **Cantidad de capas:** Es la cantidad de capas que tendrá la red neuronal. Tendremos una capa input, x ocultas y una capa de salida.
- **Cantidad de neuronas por capa:** Tendremos una neurona en la capa de salida porque queremos predecir un valor numerico. En la capa de entrada tendremos 8 neuronas, la cantidad de features que tiene el dataset.
- **Optimizador:** Un optimizador es una implementación concreta del algoritmo de descenso por gradiente dentro de backpropagation.
    - SGD (Stochastic Gradient Descent): Backpropagation simple, sin ningún tipo de optimización. 
    - Adam (Adaptive moment estimation): Hace un seguimiento de una media de decaimiento exponencial de gradientes pasados y de gradientes cuadrados pasados.
- **Función de perdida:** Es la función utilizada por el optimizador para minimizar. En regresion utilizaremos Mean Absolute Error (MAE)
- **Learning rate o tasa de aprendizaje:** Indica qué tan rápido se va descediendo en la función de costo, qué tan rápido va acercandose al minimo local o global.
- **Metricas:** Se van calculando en cada iteracion o batch. Utilizaremos Mean Squared Error (MSE).
- **Cantidad de epochs:** Es la cantidad de iteraciones que tendrá el entrenamiento.

Ejecutamos una copia de los modelos

In [3]:
x_train = _x_train_model1.copy()
y_train = _y_train_model1.copy()
x_test = _x_test_model1.copy()
y_test = _y_test_model1.copy()

In [4]:
x_train.head()

,latitud,longitud,ambientes,dormitorios,superficie_cubierta,comuna,Departamento,PH
0,-34.583010,-58.481533,4.0,3.0,70.0,15,1.0,0.0
1,-34.660079,-58.485894,6.0,4.0,275.0,9,0.0,0.0
2,-34.581856,-58.425088,3.0,2.0,97.0,14,1.0,0.0
3,-34.607547,-58.442194,2.0,1.0,42.0,6,1.0,0.0
4,-34.607416,-58.509680,2.0,1.0,38.0,11,1.0,0.0


In [5]:
y_train.head()

,pxm2
0,2911.392405
1,671.428571
2,2546.391753
3,1733.333333
4,2804.878049


In [6]:
x_train.shape

(58133, 8)

+ Transformo los set de datos

In [7]:
sscaler = StandardScaler()
sscaler.fit(pd.DataFrame(x_train))

StandardScaler()

In [8]:
x_train_transform = sscaler.transform(pd.DataFrame(x_train))
x_test_transform = sscaler.transform(pd.DataFrame(x_test))

In [9]:
x_train_transform

array([[ 0.50172268, -1.12215239,  0.89519544, ...,  1.27157065,
         0.40756802, -0.34041477],
       [-2.41244287, -1.23741004,  2.38416976, ...,  0.04401013,
        -2.45357818, -0.34041477],
       [ 0.54533212,  0.36963696,  0.15070829, ...,  1.06697723,
         0.40756802, -0.34041477],
       ...,
       [-0.42789946, -0.17643136, -0.59377887, ..., -0.56977013,
         0.40756802, -0.34041477],
       [-2.51066718, -1.91003212,  1.6396826 , ...,  0.04401013,
        -2.45357818,  2.93759284],
       [-0.08062026, -2.23160042,  0.89519544, ...,  0.45319697,
        -2.45357818,  2.93759284]])

## Modelo 1

In [19]:
d_in=8
d_out=1

modelo_1 = keras.Sequential([
    keras.layers.Dense(8,input_shape=(d_in,), activation="relu"),
    
    keras.layers.Dense(d_out, )])

In [20]:
modelo_1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 8)                 72        
                                                                 
 dense_8 (Dense)             (None, 1)                 9         
                                                                 
Total params: 81
Trainable params: 81
Non-trainable params: 0
_________________________________________________________________


In [22]:
modelo_1.compile(
      optimizer=keras.optimizers.SGD(learning_rate=0.01), 
      loss='mae', 
      metrics=['mse'], 
)

In [23]:
# Entrenamiento del modelo
modelo_1.fit(x_train_transform,y_train,epochs=50,batch_size=10,verbose=2)

Epoch 1/50
5814/5814 - 4s - loss: 665.7269 - mse: 1127414.2500 - 4s/epoch - 754us/step
Epoch 2/50
5814/5814 - 5s - loss: 518.6342 - mse: 679279.3750 - 5s/epoch - 919us/step
Epoch 3/50
5814/5814 - 4s - loss: 510.7743 - mse: 659503.8125 - 4s/epoch - 734us/step
Epoch 4/50
5814/5814 - 4s - loss: 508.2623 - mse: 654783.8750 - 4s/epoch - 706us/step
Epoch 5/50
5814/5814 - 4s - loss: 507.0320 - mse: 655059.4375 - 4s/epoch - 726us/step
Epoch 6/50
5814/5814 - 5s - loss: 506.6178 - mse: 655167.8750 - 5s/epoch - 855us/step
Epoch 7/50
5814/5814 - 5s - loss: 506.2896 - mse: 654720.6250 - 5s/epoch - 817us/step
Epoch 8/50
5814/5814 - 5s - loss: 505.6833 - mse: 651325.9375 - 5s/epoch - 794us/step
Epoch 9/50
5814/5814 - 5s - loss: 505.1441 - mse: 651166.9375 - 5s/epoch - 804us/step
Epoch 10/50
5814/5814 - 4s - loss: 504.4974 - mse: 648332.0000 - 4s/epoch - 757us/step
Epoch 11/50
5814/5814 - 5s - loss: 504.1756 - mse: 646986.0000 - 5s/epoch - 775us/step
Epoch 12/50
5814/5814 - 4s - loss: 502.6363 - mse: 

In [24]:
y_pred = modelo_1.predict(x_test_transform)

mae = mean_absolute_error(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred)

print(f"Error absoluto medio {mae}")
print(f"Error cuadrático medio {mse}")

455/455 [==============================] - 0s 808us/step
Error absoluto medio 2632.6389761478313
Error cuadrático medio 10640431.25052853


## Modelo 2

In [25]:
d_in=8
d_out=1

modelo_2 = keras.Sequential([
    keras.layers.Dense(8,input_shape=(d_in,), activation="relu"),
    
    keras.layers.Dense(2, activation="tanh" ),
    
    keras.layers.Dense(d_out, )])

In [26]:
modelo_2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 8)                 72        
                                                                 
 dense_10 (Dense)            (None, 2)                 18        
                                                                 
 dense_11 (Dense)            (None, 1)                 3         
                                                                 
Total params: 93
Trainable params: 93
Non-trainable params: 0
_________________________________________________________________


In [27]:
modelo_2.compile(
      optimizer=keras.optimizers.SGD(learning_rate=0.01), 
      loss='mae', 
      metrics=['mse'], 
)

In [28]:
# Entrenamiento del modelo
modelo_2.fit(x_train_transform,y_train,epochs=50,batch_size=10,verbose=2)

Epoch 1/50
5814/5814 - 5s - loss: 2270.2227 - mse: 6124504.0000 - 5s/epoch - 900us/step
Epoch 2/50
5814/5814 - 5s - loss: 2095.9065 - mse: 5363692.5000 - 5s/epoch - 842us/step
Epoch 3/50
5814/5814 - 5s - loss: 1921.8785 - mse: 4662857.5000 - 5s/epoch - 775us/step
Epoch 4/50
5814/5814 - 5s - loss: 1749.1750 - mse: 4026891.2500 - 5s/epoch - 800us/step
Epoch 5/50
5814/5814 - 5s - loss: 1579.8099 - mse: 3455174.0000 - 5s/epoch - 863us/step
Epoch 6/50
5814/5814 - 4s - loss: 1416.9655 - mse: 2952105.2500 - 4s/epoch - 750us/step
Epoch 7/50
5814/5814 - 5s - loss: 1264.6191 - mse: 2519243.7500 - 5s/epoch - 780us/step
Epoch 8/50
5814/5814 - 5s - loss: 1126.9596 - mse: 2156782.0000 - 5s/epoch - 800us/step
Epoch 9/50
5814/5814 - 5s - loss: 1007.7740 - mse: 1863271.8750 - 5s/epoch - 806us/step
Epoch 10/50
5814/5814 - 5s - loss: 910.0781 - mse: 1633937.5000 - 5s/epoch - 809us/step
Epoch 11/50
5814/5814 - 5s - loss: 834.1715 - mse: 1459187.6250 - 5s/epoch - 783us/step
Epoch 12/50
5814/5814 - 4s - los

In [29]:
y_pred = modelo_2.predict(x_test_transform)

mae = mean_absolute_error(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred)

print(f"Error absoluto medio {mae}")
print(f"Error cuadrático medio {mse}")

455/455 [==============================] - 0s 801us/step
Error absoluto medio 869.3670241384706
Error cuadrático medio 1480091.557447561


Disminuyo el error

## Modelo 3

In [30]:
d_in=8
d_out=1

modelo_3 = keras.Sequential([
    keras.layers.Dense(8,input_shape=(d_in,), activation="relu"),
    
    keras.layers.Dense(2, activation="tanh" ),
    
    keras.layers.Dense(d_out, )])

In [31]:
modelo_3.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 8)                 72        
                                                                 
 dense_13 (Dense)            (None, 2)                 18        
                                                                 
 dense_14 (Dense)            (None, 1)                 3         
                                                                 
Total params: 93
Trainable params: 93
Non-trainable params: 0
_________________________________________________________________


In [32]:
modelo_3.compile(
      optimizer=keras.optimizers.SGD(learning_rate=0.01), 
      loss='mae', 
      metrics=['mse'], 
)

In [33]:
# Entrenamiento del modelo
modelo_3.fit(x_train_transform,y_train,epochs=100,batch_size=16,verbose=2)

Epoch 1/100
3634/3634 - 3s - loss: 2301.4167 - mse: 6265573.5000 - 3s/epoch - 884us/step
Epoch 2/100
3634/3634 - 3s - loss: 2192.4392 - mse: 5776061.5000 - 3s/epoch - 801us/step
Epoch 3/100
3634/3634 - 3s - loss: 2083.4766 - mse: 5309455.5000 - 3s/epoch - 764us/step
Epoch 4/100
3634/3634 - 3s - loss: 1974.6259 - mse: 4867917.5000 - 3s/epoch - 789us/step
Epoch 5/100
3634/3634 - 3s - loss: 1866.1239 - mse: 4450082.5000 - 3s/epoch - 767us/step
Epoch 6/100
3634/3634 - 3s - loss: 1758.3347 - mse: 4057627.0000 - 3s/epoch - 752us/step
Epoch 7/100
3634/3634 - 3s - loss: 1651.7354 - mse: 3690412.2500 - 3s/epoch - 748us/step
Epoch 8/100
3634/3634 - 3s - loss: 1547.0867 - mse: 3348864.5000 - 3s/epoch - 785us/step
Epoch 9/100
3634/3634 - 3s - loss: 1445.2424 - mse: 3034706.7500 - 3s/epoch - 877us/step
Epoch 10/100
3634/3634 - 3s - loss: 1347.2334 - mse: 2748592.2500 - 3s/epoch - 855us/step
Epoch 11/100
3634/3634 - 3s - loss: 1253.9591 - mse: 2489571.5000 - 3s/epoch - 856us/step
Epoch 12/100
3634/3

In [34]:
y_pred = modelo_3.predict(x_test_transform)

mae = mean_absolute_error(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred)

print(f"Error absoluto medio {mae}")
print(f"Error cuadrático medio {mse}")

455/455 [==============================] - 0s 737us/step
Error absoluto medio 642.9030507443471
Error cuadrático medio 916732.3549775477


## Modelo 4

In [36]:
d_in=8
d_out=1

modelo_4 = keras.Sequential([
    keras.layers.Dense(8,input_shape=(d_in,), activation="relu"),
    
    keras.layers.Dense(2, activation="tanh" ),
    
    keras.layers.Dense(d_out, )])

In [37]:
modelo_4.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 8)                 72        
                                                                 
 dense_19 (Dense)            (None, 2)                 18        
                                                                 
 dense_20 (Dense)            (None, 1)                 3         
                                                                 
Total params: 93
Trainable params: 93
Non-trainable params: 0
_________________________________________________________________


In [38]:
modelo_4.compile(
      optimizer='adam', 
      loss='mae', 
      metrics=['mse'], 
)

In [39]:
# Entrenamiento del modelo
modelo_4.fit(x_train_transform,y_train,epochs=100,batch_size=16,verbose=2)

Epoch 1/100
3634/3634 - 4s - loss: 2347.5730 - mse: 6479163.0000 - 4s/epoch - 1ms/step
Epoch 2/100
3634/3634 - 3s - loss: 2336.4302 - mse: 6426984.0000 - 3s/epoch - 881us/step
Epoch 3/100
3634/3634 - 3s - loss: 2325.5286 - mse: 6376093.0000 - 3s/epoch - 863us/step
Epoch 4/100
3634/3634 - 3s - loss: 2314.6184 - mse: 6325543.5000 - 3s/epoch - 863us/step
Epoch 5/100
3634/3634 - 3s - loss: 2303.7192 - mse: 6275179.0000 - 3s/epoch - 849us/step
Epoch 6/100
3634/3634 - 3s - loss: 2292.8267 - mse: 6225100.5000 - 3s/epoch - 876us/step
Epoch 7/100
3634/3634 - 3s - loss: 2281.9260 - mse: 6175266.5000 - 3s/epoch - 840us/step
Epoch 8/100
3634/3634 - 3s - loss: 2271.0376 - mse: 6125613.5000 - 3s/epoch - 862us/step
Epoch 9/100
3634/3634 - 3s - loss: 2260.1448 - mse: 6076222.0000 - 3s/epoch - 840us/step
Epoch 10/100
3634/3634 - 3s - loss: 2249.2463 - mse: 6027136.0000 - 3s/epoch - 961us/step
Epoch 11/100
3634/3634 - 3s - loss: 2238.3574 - mse: 5978249.5000 - 3s/epoch - 844us/step
Epoch 12/100
3634/363

In [40]:
y_pred = modelo_4.predict(x_test_transform)

mae = mean_absolute_error(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred)

print(f"Error absoluto medio {mae}")
print(f"Error cuadrático medio {mse}")

455/455 [==============================] - 0s 772us/step
Error absoluto medio 1263.1427911532794
Error cuadrático medio 2452647.5115254335
